### Homework3

In [1]:
import numpy
import urllib
import scipy.optimize
import random
import json
import random
import gzip
import pandas 

In [2]:
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

### Tasks (Purchase prediction)

### 0.split the training data 

In [3]:
cd assignment1

/home/rstudio/Desktop/MSBA/FALL /CSE258 Recommended System and Web Mining/assignment 1/assignment1


In [4]:
data = list(readGz("train.json.gz"))
data_1 = data

In [5]:
random.shuffle(data_1)
train = data_1[:100000]
validation = data_1[100000:]

In [7]:
train[1]

{'reviewTime': '12 24, 2012',
 'reviewText': "As always, very happy with my kenneth cole purchase,the peacoat is high quality, fashionable fit, buttons are solid, the bib easily unzips, which is good because i prefer not to have one and use a scarf if its super cold. Im 6'1 180 lb and ordered a medium, which is slightly big but a small would be too small. The sleeve length is perfect though. The coat works perfect with jeans and casual as it does dress pants and suit jacket.",
 'helpful': {'nHelpful': 4, 'outOf': 5},
 'reviewerID': 'U360404137',
 'reviewHash': 'R772706618',
 'categories': [['Clothing, Shoes & Jewelry', 'K', 'Kenneth Cole New York'],
  ['Clothing, Shoes & Jewelry',
   'Men',
   'Clothing',
   'Jackets & Coats',
   'Wool & Blends']],
 'unixReviewTime': 1356307200,
 'itemID': 'I391197657',
 'rating': 5.0,
 'summary': 'Very nice peacoat!',
 'categoryID': 1}

### 1.Build such a set by randomly sampling users and items until you have 100,000 non-purchased user/item pairs.Evaluate the performance (accuracy) of the baseline model on the validation set you have built.

In [6]:
## positive sample 
validation_positive = []
for l in validation:
        validation_positive.append(l['reviewerID']+'-'+l['itemID'])
validation_positive[1]

'U470297302-I490884580'

In [7]:
## pairs existed
allReviewerID=[]
allItemID=[]

for i in data_1:
    allReviewerID.append(i['reviewerID'])
    allItemID.append(i['itemID'])
    
set_allR = list(set(allReviewerID))
set_allI = list(set(allItemID))

purchase_pair = {}
for content in data_1:
    if purchase_pair.get(content['reviewerID']):
        purchase_pair[content['reviewerID']].append(content['itemID'])
    else:
        purchase_pair[content['reviewerID']] = [content['itemID']]


In [8]:
## negative samples 
count = 0
validation_negative = {}
while count < len(train):
    rid = set_allR[random.randint(0,len(set_allR)-1)]
    iid = set_allI[random.randint(0,len(set_allI)-1)]
    if iid in purchase_pair[rid]:
        count = count
    elif iid not in purchase_pair[rid]:
        if (validation_negative.get(rid)):
            if iid in validation_negative[rid]:
                count = count
            else:
                validation_negative[rid].append(iid)
                count += 1
        else:
            validation_negative[rid]= [iid]
            count += 1  
            

In [9]:
## negative samples to list

validation_negative_list = []

for i in validation_negative.keys():
    if len(validation_negative[i]) > 1:
        for ii in validation_negative[i]:
            validation_negative_list.append([i,ii])
    else:
        validation_negative_list.append([i,validation_negative[i][0]])  

In [8]:
itemCount = defaultdict(int)
totalPurchases = 0

for l in train:
    user,item = l['reviewerID'],l['itemID']
    itemCount[item] += 1
    totalPurchases += 1

In [11]:
mostPopular = [(itemCount[x], x) for x in itemCount]
mostPopular.sort()
mostPopular.reverse()

In [12]:
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/2: break

In [13]:
count = 0
for l in validation_positive:
    u,i = l.strip().split('-')
    if i in return1:
        count += 1

for r, i in validation_negative_list:
    if not i in return1:
        count = count +1
        
print('accuracy for the baseline code is ' + str(count / 200000))

accuracy for the baseline code is 0.630145


### 2.The existing ‘purchase prediction’ baseline just returns True if the item in question is ‘popular,’ using a threshold of the 50th percentile of popularity (totalPurchases/2). Assuming that the ‘non-purchased’ test examples are a random sample of user-purchase pairs, is this particular threshold value the best? If not, see if you can find a better one (and report its performance), or if so, explain why it is the best (1 mark).

In [14]:
accuracy = {}
for n in numpy.arange(0, 100, 1):     
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalPurchases * (n/100) : break
    count_1 = 0 
    for l in validation_positive:
        u,i = l.strip().split('-')
        if i in return1:
            count_1 += 1
    for r, i in validation_negative_list:
        if not i in return1:
            count_1 = count_1 +1
    accuracy[n] = count_1 / 200000


In [15]:
max(accuracy,key=accuracy.get)

54

When the threshold is 54%, the accracy is max.

### 3.Users may tend to repeatedly purchase items of the same type. Build a baseline that returns ‘True’ if a user has purchased an item of the same category before (at least one category in common), or zero otherwise (1 mark).


In [25]:
## pairs existed

trainReviewerID=[]
trainCategory={}

for i in data_1:
    trainReviewerID.append(i['reviewerID'])
    
set_trainR = list(set(trainReviewerID))

for content in data_1:
    if trainCategory.get(content['reviewerID']):
        for u in content['categories']:
            trainCategory[content['reviewerID']].append(u)
    else:
        temp = []
        for u in content['categories']:
            temp.append(u)
            trainCategory[content['reviewerID']] = temp
    

In [12]:
item_categories = defaultdict(list)
for i in data_1:
    item = i['itemID']
    categories = i['categories']
    item_categories[item] = categories

In [18]:
count_2 = 0 
for l in validation_positive:
    u,i = l.strip().split('-')
    if trainCategory.get(u):
        if sum([c in trainCategory[u] for c in item_categories[i]]) > 0 :
            count_2 += 1
        else: count_2 = count_2
        
for u,i in validation_negative_list:
    if trainCategory.get(u):
        if sum([c in trainCategory[u] for c in item_categories[i]]) == 0 :
            count_2 += 1 
        else: count_2 = count_2

In [19]:
print('accuracy for the baseline code is ' + str(count_2 / 200000))

accuracy for the baseline code is 0.558805


### 4.To run our model on the test set, we’ll have to use the files ‘pairs Purchase.txt’ to find the review- erID/itemID pairs about which we have to make predictions. Using that data, run the above model and upload your solution to Kaggle. Tell us your Kaggle user name (1 mark). If you’ve already uploaded a better solution to Kaggle, that’s fine too!

In [23]:
predictions = open("predictions_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if trainCategory.get(u):
        if sum([c in trainCategory[u] for c in item_categories[i]]) > 0 :
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
    else: predictions.write(u + '-' + i + ",0\n")

predictions.close()

My id in kaggle is ‘Lu Yao’, and the accuracy is 0.59971.

### Tasks (Rating prediction)

### 5.What is the performance of a trivial predictor

In [6]:
allRatings = []

for l in train:
    allRatings.append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)

In [7]:
globalAverage

4.23691

In [8]:
predictions = []
rid = []
iid = []
rating = []
predict = []
for l in validation:
    u,i,r = l['reviewerID'],l['itemID'],l['rating']
    rid.append(u)
    iid.append(i)
    rating.append(r)
    predict.append(globalAverage)

    
predictions  = set(zip(rid,iid,rating,predict))

In [9]:
from sklearn.metrics import mean_squared_error
mean_squared_error(rating, predict)

1.2284284599000002

### 6.Fit a predictor of the form rating(user, item) ≃ α + βuser + βitem, by fitting the mean and the two bias terms as described in the lecture notes. Use a regularization parameter of λ = 1. Report the MSE on the validation set (1 mark).

In [13]:
# calculate initial value of $\beta_{user}$ and $\beta_{item}$

userRatings = defaultdict(list)
itemRatings = defaultdict(list)
index = 0
alpha = globalAverage

for l in data_1:
    user,item = l['reviewerID'],l['itemID']
    userRatings[user].append(l["rating"])
    itemRatings[item].append(l["rating"])

userAverage = defaultdict(float)
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
itemAverage = defaultdict(float)
for b in itemRatings:
    itemAverage[b] = sum(itemRatings[b]) / len(itemRatings[b])

initial value of $\alpha$ is globalAverage, initial value of $\beta_{user}$ is userAverage, initial value of $\beta_{item}$ is itemAverage

In [11]:
def converge(value, value_last, epsilon):
    error = 0
    for i in value:
        error += abs(value[i] - value_last[i])
    #print(error)
    if error < epsilon:
        return True
    else:
        return False

In [12]:
        # update userAverage
        num_item = defaultdict(int)
        for user in userAverage:
            userAverage[user] = 0.0
        for l in data:
            user,item = l['reviewerID'],l['itemID']
            num_item[user] += 1
            userAverage[user] += (l["rating"] - globalAverage - itemAverage_last[item])
        for user in userAverage:
            userAverage[user] /= (lam + num_item[user])

NameError: name 'itemAverage_last' is not defined

In [16]:
def update(globalAverage, userAverage, itemAverage, lam = 1, epsilon = 1e-5):
    finish = False
    globalAverage_last = globalAverage
    userAverage_last = userAverage
    itemAverage_last = itemAverage
    while not finish:
        # update globalAverage
        num_train = len(data)
        globalAverage = 0.0
        for l in data_1:
            user,item = l['reviewerID'],l['itemID']
            globalAverage += (l["rating"] - userAverage_last[user] - 
                              itemAverage_last[item])
        globalAverage /= num_train
        
        # update userAverage
        num_item = defaultdict(int)
        for user in userAverage:
            userAverage[user] = 0.0
        for l in data_1:
            user,item = l['reviewerID'],l['itemID']
            num_item[user] += 1
            userAverage[user] += (l["rating"] - globalAverage - itemAverage_last[item])
        for user in userAverage:
            userAverage[user] /= (lam + num_item[user])
        
        # update itemAverage
        num_user = defaultdict(int)
        for item in itemAverage:
            itemAverage[item] = 0.0
        for l in data_1:
            user,item = l['reviewerID'],l['itemID']
            num_user[item] += 1
            itemAverage[item] += (l["rating"] - globalAverage - userAverage[user])
        for item in itemAverage:
            itemAverage[item] /= (lam + num_user[item])
        
        #print ((globalAverage - globalAverage_last)**2)
        if abs(globalAverage - globalAverage_last) < epsilon and converge(userAverage, userAverage_last, epsilon) and converge(itemAverage, itemAverage_last, epsilon):
            finish = True
        else:
            globalAverage_last = globalAverage
            userAverage_last = userAverage
            itemAverage_last = itemAverage
    return globalAverage, userAverage, itemAverage

In [ ]:
globalAverage_new, userAverage_new, itemAverage_new = update(globalAverage, userAverage, itemAverage, lam = 1, epsilon = 1e-5)

In [17]:
def cal_mse(globalAverage_new, userAverage_new, itemAverage_new):
    mse = 0.0
    for l in data_1:
        user,business = l['reviewerID'],l['itemID']
        mse += (l["rating"]-globalAverage_new-userAverage_new[user]-\
                itemAverage_new[business])**2
    mse /= 100000
    print ("mean square error on validation data is " + str(mse))

In [35]:
cal_mse(globalAverage_new, userAverage_new, itemAverage_new)

mean square error on validation data is 0.6839153332902849


### 7.Report the user and item IDs that have the largest and smallest values of β (1 mark).

In [40]:
userAverage_new_sort = sorted(userAverage_new.items(), key = lambda x: x[1])

In [43]:
userAverage_new_sort[0]

('U933497871', -2.5253677137173898)

In [44]:
userAverage_new_sort[-1]

('U605818049', 1.4494614247226256)

In [45]:
itemAverage_new_sort = sorted(itemAverage_new.items(), key = lambda x: x[1])

In [46]:
itemAverage_new_sort[0]

('I119256103', -2.399751075797392)

In [47]:
itemAverage_new_sort[-1]

('I043564971', 1.3742619052625877)

In [77]:
print ("user id with maximum beta is",userAverage_new_sort[-1])

print ("user id with minimum beta is",userAverage_new_sort[0])

print ("business id with maximum beta is",itemAverage_new_sort[-1])

print ("business id with minimum beta is",itemAverage_new_sort[0])


user id with maximum beta is ('U605818049', 1.4494614247226256)
user id with minimum beta is ('U933497871', -2.5253677137173898)
business id with maximum beta is ('I043564971', 1.3742619052625877)
business id with minimum beta is ('I119256103', -2.399751075797392)


### 8.Find a better value of λ using your validation set. Report the value you chose, its MSE, and upload your solution to Kaggle by running it on the test data (1 mark).

In [19]:
#lams = numpy.linspace(0,1,10)
lams = [0.01, 0.1, 10, 100]
for lam in lams:
    globalAverage_new1, userAverage_new1, itemAverage_new1 = update(globalAverage, userAverage, itemAverage, lam = lam, epsilon = 1e-6)
    print ("lambda is " + str(lam))
    cal_mse(globalAverage_new1, userAverage_new1, itemAverage_new1)

KeyboardInterrupt: 

In [18]:
lams = [1,2,3,4,5,6,7,8,9,10]

    
for lam in lams:
    globalAverage_new1, userAverage_new1, itemAverage_new1 = update(globalAverage, userAverage, itemAverage, lam = lam, epsilon = 1e-6)
    print ("lambda is " + str(lam))
    cal_mse(globalAverage_new1, userAverage_new1, itemAverage_new1)

lambda is 1
mean square error on validation data is 1.364446495110649
lambda is 2
mean square error on validation data is 1.4270198088388026
lambda is 3
mean square error on validation data is 1.4873165137070894
lambda is 4
mean square error on validation data is 1.5419956240818535


KeyboardInterrupt: 

In [39]:

predictions = open("predictions_Rating.txt", 'w')

for l in open("pairs_Rating.txt"):
    if l.startswith("revierweID"):
        #header
        predictions.write(l)
        continue
        
    user,item = l.strip().split('-')
    predictions.write(user + '-' + item + ',' + \
                str(globalAverage_new1 + userAverage_new1[user]\
                    +itemAverage_new1[item]) + '\n')

predictions.close()